In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess
import importlib.util
import pandas as pd
import openai
import nest_asyncio
import asyncio

nest_asyncio.apply()

# Set OpenAI API key
with open('/content/drive/MyDrive/key.txt', 'r') as file:
    api_key = file.read().strip()
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = api_key

# Paths
folder_path = '/content/drive/MyDrive/Programs/'
csv_input_path = os.path.join(folder_path, 'program_rmse_results.csv')

# Check if results file exists and read existing results
if os.path.exists(csv_input_path):
    results_df = pd.read_csv(csv_input_path)
    evaluated_programs = set(results_df['code'].tolist())
else:
    results_df = pd.DataFrame(columns=['code', 'prompt', 'RMSE'])
    evaluated_programs = set()

# List all Python files in the folder
program_files = [f for f in os.listdir(folder_path) if f.endswith('.py')]

results = []

# Execute only unevaluated Python files
for file_name in program_files:
    if file_name in evaluated_programs:
        print(f"Skipping already evaluated program: {file_name}")
        continue

    file_path = os.path.join(folder_path, file_name)
    module_name = os.path.splitext(file_name)[0]
    prompt_file = module_name + '.txt'

    # Dynamically import module
    spec = importlib.util.spec_from_file_location(module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    # Execute CalculateRMSE and handle potential errors
    try:
        rmse = module.CalculateRMSE()
        rmse_numeric = float(rmse)
        results_df = pd.concat([results_df, pd.DataFrame([{'code': file_name, 'prompt': prompt_file, 'RMSE': rmse_numeric}])], ignore_index=True)
        print(f"Executed {file_name} successfully. RMSE: {rmse_numeric:.4e}")
    except Exception as e:
        print(f"Error executing {file_name}: {e}")

# Sort results by RMSE
results_df.sort_values(by='RMSE', inplace=True)

# Save sorted results to CSV on Google Drive
csv_output_path = os.path.join(folder_path, 'program_rmse_results.csv')
results_df.to_csv(csv_output_path, index=False, encoding='utf-8', float_format='%.4e')
print("Results saved to program_rmse_results.csv")

if not results_df.empty:
    best_program = results_df.iloc[0]
    print(f"\nBest Program: {best_program['code']} with RMSE: {best_program['RMSE']:.4e}")
else:
    print("\nNo successful executions.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
RMSE: 1.0205539627036521e-05
R² score: 0.48159106460857704
Executed program1.py successfully. RMSE: 1.0206e-05


<ipython-input-2-d9a8e167c77b>:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{'code': file_name, 'prompt': prompt_file, 'RMSE': rmse_numeric}])], ignore_index=True)


Executed program2.py successfully. RMSE: 1.4333e-05


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
RMSE: 0.07309275478156613
R² Score: -26591888.18346646
Executed program3.py successfully. RMSE: 7.3093e-02
Executed program4.py successfully. RMSE: 1.4333e-05
Executed program5.py successfully. RMSE: 1.4333e-05
RMSE: 1.43e-05
R² Score: -0.02
Executed program6.py successfully. RMSE: 1.4333e-05
Results saved to program_rmse_results.csv

Best Program: program1.py with RMSE: 1.0206e-05
